In [1]:
################################################################################
#Setup
################################################################################

#we may need some code in the ../python directory and/or matplotlib styles
import sys
import os
sys.path.append('../../python/')

#set up matplotlib
os.environ['MPLCONFIGDIR'] = '../../mplstyles'
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('standard')

#fonts
# Set the font dictionaries (for plot title and axis titles)
title_font = {'fontname':'Arial', 'size':'16', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'32'}
legend_font = {'fontname':'Arial', 'size':'22'}

#fonts global settings
mpl.rc('font',family=legend_font['fontname'])

import numpy as np
import pandas as pd
import constants as const
from scipy.optimize import minimize

#For some reason, this speeds things up!
os.environ["OMP_NUM_THREADS"] = "1"

#Added for notebook
import emcee

In [2]:
################################################################################
#Fit Settings
################################################################################
#Construct a dictionary to store all the MCMC fit parameters and results
#These are all the settings a user will regularly change

        ########################## Data Settings ##########################
mcmc_data={'g4_load_frac':0.1,
           'cap_load_frac':0.1,
           'cap_sim_file':'../data/v3_400k.pkl',
           'cap_rcapture':0.218,
           ########################## Spectrum Settings ##########################
           'Emax': 2000, #[eVee]
           'Ebins': np.linspace(0,2500,251), #np.linspace(0,2000,201),
           'Efit_min':50, #[eVee]
           'Efit_max':2000, #1750, #[eVee]
           'spectrum_units':'reco-rate', #One of {'counts', 'reco-rate'}
           ########################## Yield Model Settings ##########################
           #'Y_model':'Lind',
           #'Y_labels': [r'k', r'$F_{NR}$'],
           #'Y_bounds': [(0.05,0.3),(0,30)],
           #'Y_model':'Chav',
           #'Y_labels': [r'k', r'$a^{-1}$', r'$F_{NR}$'],
           #'Y_bounds': [(0.05,0.3),(0,2e3),(0,30)],
           'Y_model':'Sor',
           'Y_labels': [r'k', r'q', r'$F_{NR}$'],
           'Y_bounds': [(0.05,0.3),(0,3e-2),(0,30)],
           #'Y_model':'AC',
           #'Y_labels': [r'k', r'$\xi$', r'$F_{NR}$'],
           #'Y_bounds': [(0.05,0.3),(0,2e3),(0,30)],
           ########################## Sim Spectra Settings ##########################
           'ER_spec_model':'sim', #One of {'sim', 'flat') to selct from G4 simulation or flat
           'ER_par_labels':[r'$scale_{ER}$'],
           'ER_par_bounds':[(0,20)], #Unitless scaling factor
           'NR_spec_model':'sim', #One of {'sim', 'flat', 'exp') to selct from G4 simulation, flat, or exponential
           'NR_par_labels':[r'$scale_{NR}$'],
           'NR_par_bounds':[(0,20)], #Unitless scaling factor
           'NG_spec_model':'sim', #Not going to implement anything other than sim for (n,gamma) yet
           'NG_par_labels':[r'$scale_{ng}$'],
           'NG_par_bounds':[(0,10)], #Unitless scaling factor
           ########################## Likelihood Settings ##########################
           'likelihood':'SNorm', #One of {'Pois', 'Norm', 'SNorm'} Only SNorm uses sigmas, others assume Pois stats
           ########################## Uncertainty Settings ##########################
           'doDetRes': True, #Include detector resolution effects
           'fpeak':1, #0.753 -- 1.0
           'doEffsyst':False, #Include systematics from cut efficiencies
           'doBurstLeaksyst':False, #Include burst cut leakage systematic
           ########################## MCMC Settings ##########################
           'nwalkers':128,
           'nstep':500000,
           'guesses':'Uniform', #Can either be uniform or shape (nwalkers, ndim),
           'moves':'DE8020',#'Default': StretchMove, 'DE8020': 80/20 DEMove/DESnookerMove
           'saveMCMC':True
          }

In [3]:
################################################################################
#Parse Options
################################################################################
mcmc_data['labels']=mcmc_data['Y_labels']+mcmc_data['ER_par_labels']+mcmc_data['NR_par_labels']+mcmc_data['NG_par_labels']
mcmc_data['bounds']=mcmc_data['Y_bounds']+mcmc_data['ER_par_bounds']+mcmc_data['NR_par_bounds']+mcmc_data['NG_par_bounds']    
mcmc_data['ndim']=len(mcmc_data['labels'])

In [4]:
################################################################################
#Load Data
################################################################################
import R68_load as r68
import R68_spec_tools as spec
#Import likelihood functions
import likelihoods as l
import R68_efficiencies as eff

#Set eVee energy binning
Ebins_ctr=(mcmc_data['Ebins'][:-1]+mcmc_data['Ebins'][1:])/2

#Set fit range
mcmc_data['spec_bounds']=(np.digitize(mcmc_data['Efit_min'],mcmc_data['Ebins'])-1,np.digitize(mcmc_data['Efit_max'],mcmc_data['Ebins'])-1)

#Measured spectra background subtraction
meas=r68.load_measured(verbose=True)
#tlive_PuBe=meas['PuBe']['tlive']

#Note that N_meas may be either in counts or reconstructed rate, depending on mcmc_data['spectrum_units']
#uncertainty is (high,low)
N_meas,dN_meas=spec.doBkgSub(meas, mcmc_data['Ebins'], mcmc_data['Efit_min'], mcmc_data['Efit_max'],\
                             doEffsyst=mcmc_data['doEffsyst'], doBurstLeaksyst=mcmc_data['doBurstLeaksyst'],\
                             output=mcmc_data['spectrum_units'])

if mcmc_data['likelihood']=='SNorm':
    #Precalculate split normal likelihood params if we're going to need it
    SNpars=l.getSNparsArray(N_meas[slice(*mcmc_data['spec_bounds'])],dN_meas[0][slice(*mcmc_data['spec_bounds'])],dN_meas[1][slice(*mcmc_data['spec_bounds'])])
    SNpars=SNpars.T

#Load g4 Simulations
#No need for this if we're using analytical models for both
if (mcmc_data['ER_spec_model']=='sim') or (mcmc_data['NR_spec_model']=='sim'):
    g4=r68.load_G4(load_frac=mcmc_data['g4_load_frac'])

    #Trim events that won't figure into the analysis range
    #Trimmed sim data
    if (mcmc_data['ER_spec_model']=='sim'):
        Eee_er=np.sum(g4['ER']['E'],axis=1)
        Evec_er_cut=(Eee_er>10) & (Eee_er<3e3)
        Evec_er=g4['ER']['E'][Evec_er_cut]

    if (mcmc_data['NR_spec_model']=='sim'):
        Eee_nr=np.sum(g4['NR']['E'],axis=1)
        Evec_nr_cut=(Eee_nr>10) & (Eee_nr<30e3)
        Evec_nr=g4['NR']['E'][Evec_nr_cut]


#Load Capture Simulations
cap=r68.load_simcap(file=mcmc_data['cap_sim_file'], rcapture=mcmc_data['cap_rcapture'], load_frac=mcmc_data['cap_load_frac'])

#Pre-calculate Simulated ER spectrum
#This is independent of other params, so we can do this once and reuse it
if mcmc_data['ER_spec_model']=='sim':
    if mcmc_data['spectrum_units']=='reco-rate':
        N_er = spec.buildAvgSimSpectrum_ee(Ebins=mcmc_data['Ebins'], Evec=Evec_er, Yield=1.0, F=const.F, scale=1,\
                                           doDetRes=mcmc_data['doDetRes'], fpeak=mcmc_data['fpeak'],\
                                           doEffs=False)
else:
    print('ER_spec_model: ',mcmc_data['ER_spec_model'],' not implemented yet.')
        
#Import yield models
import R68_yield as Yield
#Initialize Yield model
Y=Yield.Yield('Lind',[0.15])
model=mcmc_data['Y_model']
Y=Yield.Yield(model,np.zeros(Y.model_npar[model]))

Loading Measured Data...
PuBe events:  (476731,)
Bkg events:  (174636,)
Loading Geant4 Data...
(528848, 7)
(129555, 7)


../../python/R68_spec_tools.py:55: RuntimeWarning: divide by zero encountered in true_divide
  (eff.dcutEffFit_bkg(Ebins_ctr)/eff.cutEffFit_bkg(Ebins_ctr))**2 +\
../../python/R68_spec_tools.py:60: RuntimeWarning: divide by zero encountered in true_divide
  R_meas_Bkg = N_meas_Bkg/TE_Bkg
../../python/R68_spec_tools.py:62: RuntimeWarning: invalid value encountered in true_divide
  dR_meas_PuBe_hi = R_meas_PuBe*np.sqrt( (dN_meas_PuBe_Pois/N_meas_PuBe)**2 +\
../../python/R68_spec_tools.py:65: RuntimeWarning: invalid value encountered in true_divide
  doBurstLeaksyst*(eff.dtrigburstLeak(Ebins_ctr)/N_meas_PuBe)**2 +\
../../python/R68_spec_tools.py:65: RuntimeWarning: divide by zero encountered in true_divide
  doBurstLeaksyst*(eff.dtrigburstLeak(Ebins_ctr)/N_meas_PuBe)**2 +\
../../python/R68_spec_tools.py:65: RuntimeWarning: invalid value encountered in multiply
  doBurstLeaksyst*(eff.dtrigburstLeak(Ebins_ctr)/N_meas_PuBe)**2 +\
../../python/R68_spec_tools.py:68: RuntimeWarning: invalid valu

Loading NRs...
0.0  min
Loading ERs...
0.0  min
Loading (n,gamma) Data...
382155
dict_keys(['xx', 'yy', 'ex', 'ey'])


In [5]:
################################################################################
#Calculate Log probability, log(likelihood*prior)
################################################################################
#theta: array of fit parameters (yield_par0, yield_par1, ...,  F_NR, scale_g4, scale_ng, ...)
#    These are in the same order as mcmc_data['labels']
#theta_bounds: paramter bounds, shape should be len(theta)x2
#spec_bounds: range of bin numbers in spectrum to consider. The analysis range is [bin_low,bin_high)
#likelihood: Likelihood function, either 'Pois' or 'Norm'

N_pred = None #Setting this up so I can make it global to call it in a moment.

def calc_log_likelihood(theta=[0.2,1,1,1,1,1], theta_bounds=tuple(mcmc_data['bounds']), spec_bounds=mcmc_data['spec_bounds'],
                  likelihood=mcmc_data['likelihood'],verbose=False,f_ngzero=False): 
    
    #Access the global data
    #These must be already defined!!!
    global mcmc_data, eff, N_meas, N_er, Evec_nr, cap, Y, N_pred
    
    if f_ngzero:
        theta[-1] = 0
    
    ############
    #Parse fit params
    nYpar=Y.npars

    Y.set_pars(theta[:nYpar])
    F_NR=theta[nYpar] #theta[2] 
    if verbose:
        print(Y.pars,F_NR)
    
    if not Y.solve():
        return -np.inf
        
    #Grab correct ER scaling factor from theta
    if mcmc_data['ER_spec_model']=='sim':
        if '$scale_{G4}$' in mcmc_data['labels']:
            scale_er=theta[mcmc_data['labels'].index('$scale_{G4}$')]/g4['ER']['tlive']
        else:
            scale_er=theta[mcmc_data['labels'].index('$scale_{ER}$')]/g4['ER']['tlive']
        
    ##########
    #Build the spectra    
    if mcmc_data['spectrum_units']=='reco-rate':
        #Don't apply any efficiency effects to simulated spectrum
        #NR
        if mcmc_data['NR_spec_model']=='sim':
            N_nr=spec.buildAvgSimSpectrum_ee(Ebins=mcmc_data['Ebins'], Evec=Evec_nr, Yield=Y, F=F_NR, scale=1,\
                                             doDetRes=mcmc_data['doDetRes'], fpeak=mcmc_data['fpeak'],\
                                             doEffs=False)
            if verbose:
                print("\nN_nr INPUTS")
                print("-------------")
                print("Ebins:",mcmc_data['Ebins'])
                print("Evec_nr:",Evec_nr)
                print("Yield:",Y.pars)
                print("F_NR:",F_NR)
                print("doDetRes:",mcmc_data['doDetRes'])
                print("fpeak:",mcmc_data['fpeak'],'\n')

            if '$scale_{G4}$' in mcmc_data['labels']:
                scale_nr=theta[mcmc_data['labels'].index('$scale_{G4}$')]/g4['NR']['tlive']
            else:
                scale_nr=theta[mcmc_data['labels'].index('$scale_{NR}$')]/g4['NR']['tlive']
        elif mcmc_data['NR_spec_model']=='exp':
            N_nr=np.exp(-Ebins_ctr/theta[mcmc_data['labels'].index(r'$E0_{NR}$')])
            scale_nr=theta[mcmc_data['labels'].index(r'$R0_{NR}$')]
        
        if verbose:
            print('spec_units == reco-rate\n')
            print('N_ng INPUTS')
            print('-----------')
            print('Ebins:',mcmc_data['Ebins'])
            print('Evec:',cap['E'])
            print('dEvec:',cap['dE'])
            print('Yield:',Y.pars)
            print('F:',F_NR)
            print('doDetRes:',mcmc_data['doDetRes'])
            print('fpeak:',mcmc_data['fpeak'],'\n')
        #(n,gamma)
        N_ng=spec.buildAvgSimSpectrum_ee_composite(Ebins=mcmc_data['Ebins'], Evec=cap['E'], dEvec=cap['dE'],\
                                                   Yield=Y, F=F_NR, scale=1, doDetRes=mcmc_data['doDetRes'],\
                                                   fpeak=mcmc_data['fpeak'], doEffs=False,verbose=verbose)
        scale_ng=theta[mcmc_data['labels'].index('$scale_{ng}$')]/cap['tlive']
        
        #Calculate rate (though we'll still call them N_* just to be confusing)
        N_pred = (N_nr*scale_nr + 
                  N_er*scale_er + 
                  N_ng*scale_ng)
        if verbose:
            print("N_ng:",N_ng[0:3]) 
            print("N_nr:", N_nr[0:3])
            print("F_NR:", F_NR)
            print("N_pred:",N_pred[0])

    ##########
    #Calculate the log probability = log prior + log likelihood
    ll=None
    
    if likelihood=='Norm':
        ll = ll_norm(N_meas[slice(*spec_bounds)],N_pred[slice(*spec_bounds)])
    elif likelihood=='Pois':
        ll = ll_pois(N_meas[slice(*spec_bounds)],N_pred[slice(*spec_bounds)])
    elif likelihood=='SNorm':
        #Precalculate split normal likelihood params if we're going to need it
        SNpars=l.getSNparsArray(N_meas[slice(*spec_bounds)],dN_meas[0][slice(*spec_bounds)],dN_meas[1][slice(*spec_bounds)])
        SNpars=SNpars.T
        ll = l.ll_SNorm(N_pred[slice(*spec_bounds)],*SNpars) #4 parameters
        if verbose:
            print("SNpars.T:",SNpars[:,-5:])
            print("N_pred:",N_pred[slice(*spec_bounds)][:5])
    else:
        print('Error: Bad likelihood')
        return None
    
    if not np.isfinite(ll):
        return -np.inf
    
    return ll #When optimizing, I only want to optimize the log likelihood, not of prior*likelihood.
    #So I've slightly altered this function. Also shrunk down to SNorm for ease of troubl

/home/gerudo7/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#This is more or less straight from https://emcee.readthedocs.io/en/stable/tutorials/line/
#IMPORTANT: CHECK ALL THREE LINES TO ADJUST FOR f_ngzero=True/False
#note that bounds cannot be equal; set upper bound of f_ng=1e-9 to keep at zero.

nll = lambda *args: -calc_log_likelihood(*args)#,f_ngzero=True)
initial = np.array([0.183,2.94807038e-03, 4.58431131e+00, 2.12647135e+00,7.59037558e+00, 3.99620693e+00]) #Based on R68_MCMC_plots_v2.ipynb #set initial last value to 0 if using f_ngzero=True
soln = minimize(nll,initial,method = 'SLSQP',bounds=((0,1),(0,None),(0,None),(0,None),(0,None),(0,None)))
k,q,F_NR,f_ER,f_NR,f_ng = soln.x

# Get Chisquare!

In [7]:
print("MODEL:",Y.model,"\n")

print("STATISTICS")
print("------------")

print("Log likelihood:",calc_log_likelihood(soln.x))

if mcmc_data['likelihood'] == 'SNorm':
    chi = l.chisq(N_meas[slice(*mcmc_data['spec_bounds'])],N_pred[slice(*mcmc_data['spec_bounds'])],0.5*(dN_meas[0]+dN_meas[1])[slice(*mcmc_data['spec_bounds'])])
else:
    print("Sorry, I didn't make that yet. See cell 21 of R68_MCMC_process.ipynb for chisq calc.")
    print("Also now you get an error, probably.")
    print("▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓░░░▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓░▓▓▓▓▓▓░░░▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓░▓░░░▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓▓▓▓▓▓░▓░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓░▓░▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓░▓░▓░░░▓▓▓▓▓\n▓▓▓▓▓▓░░░░░░░░░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓░▓░▓░▓▓░░░▓▓▓▓▓▓▓\n▓▓▓░░░░▓░▓░▓░▓░▓░▓░▓░▓▓▓▓▓▓▓░▓▓▓░▓░▓░▓░▓░░░▓▓▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░▓░▓░▓░▓░▓░▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░▓░▓▓▓▓▓▓▓▓▓▓▓░▓░▓░▓░▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓░░░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░▓░░░▓▓▓▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓░░░░░░░░░▓░▓░▓░▓░▓░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░▓▓▓▓▓▓▓▓▓░▓░▓░▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓░▓░░░░░░░░░░░░░░░░░▓░▓▓▓▓▓░░░▓░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓░░░░░░░░░░░░░▓▓▓▓▓▓▓░▓▓▓░░░▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓░░░░░▓▓▓░░░░░░░░░░░▓▓▓▓▓▓▓░░░▓▓\n▓▓▓▓▓▓▓▓▓░▓░░░░░░▓▓░░░░░▓▓▓░░░░░░░░░░░▓▓░░░▓▓░░░▓░\n▓▓▓▓▓▓▓▓░░░▓▓▓░░░░░░░░░░▓▓▓░░░░░░░░░░▓▓▓░░░▓▓▓▓▓░░\n▓▓▓▓▓▓░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓░░░░░▓▓\n▓▓▓▓▓▓░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓░░░░░▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░░░░▓░▓▓▓▓▓▓▓▓▓▓▓░▓░░░░░░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓\n▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓░░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓░░░▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓░▓░░░▓▓▓░░░░░░░▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░░░░░░░░░▓▓░░░░░▓▓▓▓▓▓░░░░░░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓▓▓░░░▓▓▓▓▓░░░▓▓▓░░░▓▓▓▓\n▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓▓▓▓▓▓░░░▓▓\n▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓\n▓▓▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓░░░░░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓ ")
    print("(Source: https://steamcommunity.com/groups/ascii-art/discussions/0/312265672109050684/ )")
    print("")
dof=np.diff(mcmc_data['spec_bounds'])[0]-mcmc_data['ndim']

print('Chisq:',chi)
print('Chisq/DoF:',chi/dof,"\n")

print("PARAMETERS")
print("------------")

print("k = {0:.3f}".format(k))
print("q = {0:.5e}".format(q))
print("F_NR = {0:.3f}".format(F_NR))
print("f_ER = {0:.3f}".format(f_ER))
print("f_NR = {0:.3f}".format(f_NR))
print("f_ng = {0:.3f}\n".format(f_ng))

MODEL: Sor 

STATISTICS
------------
Log likelihood: 1215.2199435995835
Chisq: 376.5959937249989
Chisq/DoF: 1.9925713953703645 

PARAMETERS
------------
k = 0.151
q = 1.78248e-03
F_NR = 4.106
f_ER = 4.178
f_NR = 5.504
f_ng = 3.528



In [8]:
# Manual statistics calculation

params          = [1.51599043e-01,1.83004975e-03,4.30602024e+00,4.18358660e+00,5.51331376e+00,3.58263625e+00]  #Provide parameters here
#likelihoodmodel = 'SNorm'        #Provide likelihood model here, but this is the only one that works.

print("MODEL:",Y.model,"\n")

print("STATISTICS")
print("------------")

print("Log likelihood:",calc_log_likelihood(params))

if mcmc_data['likelihood'] == 'SNorm':
    chi = l.chisq(N_meas[slice(*mcmc_data['spec_bounds'])],N_pred[slice(*mcmc_data['spec_bounds'])],0.5*(dN_meas[0]+dN_meas[1])[slice(*mcmc_data['spec_bounds'])])
else:
    print("Sorry, I didn't make that yet. See cell 21 of R68_MCMC_process.ipynb for chisq calc.")
    print("Also now you get an error, probably.")
    print("▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓░░░▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓░▓▓▓▓▓▓░░░▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓░▓░░░▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓▓▓▓▓▓░▓░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓░▓░▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓░▓░▓░░░▓▓▓▓▓\n▓▓▓▓▓▓░░░░░░░░░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓░▓░▓░▓▓░░░▓▓▓▓▓▓▓\n▓▓▓░░░░▓░▓░▓░▓░▓░▓░▓░▓▓▓▓▓▓▓░▓▓▓░▓░▓░▓░▓░░░▓▓▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░▓░▓░▓░▓░▓░▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░▓░▓▓▓▓▓▓▓▓▓▓▓░▓░▓░▓░▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓░░░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░▓░░░▓▓▓▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓░░░░░░░░░▓░▓░▓░▓░▓░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░▓▓▓▓▓▓▓▓▓░▓░▓░▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓░▓░░░░░░░░░░░░░░░░░▓░▓▓▓▓▓░░░▓░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓░░░░░░░░░░░░░▓▓▓▓▓▓▓░▓▓▓░░░▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓░░░░░▓▓▓░░░░░░░░░░░▓▓▓▓▓▓▓░░░▓▓\n▓▓▓▓▓▓▓▓▓░▓░░░░░░▓▓░░░░░▓▓▓░░░░░░░░░░░▓▓░░░▓▓░░░▓░\n▓▓▓▓▓▓▓▓░░░▓▓▓░░░░░░░░░░▓▓▓░░░░░░░░░░▓▓▓░░░▓▓▓▓▓░░\n▓▓▓▓▓▓░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓░░░░░▓▓\n▓▓▓▓▓▓░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓░░░░░▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░░░░▓░▓▓▓▓▓▓▓▓▓▓▓░▓░░░░░░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓\n▓▓▓▓▓▓░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓░░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓░░░▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓░▓░░░▓▓▓░░░░░░░▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░▓▓▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓░░░░░░░░░▓▓░░░░░▓▓▓▓▓▓░░░░░░░▓▓▓▓▓\n▓▓▓▓▓▓░░░▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓▓▓░░░▓▓▓▓▓░░░▓▓▓░░░▓▓▓▓\n▓▓▓░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░▓▓▓▓▓▓▓░░░▓▓\n▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓\n▓▓▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓░░░░░░░▓▓▓▓▓\n▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓ ")
    print("(Source: https://steamcommunity.com/groups/ascii-art/discussions/0/312265672109050684/ )")
    print("")
dof=np.diff(mcmc_data['spec_bounds'])[0]-mcmc_data['ndim']

print('Chisq:',chi)
print('Chisq/DoF:',chi/dof,"\n")

print("PARAMETERS")
print("------------")

print("k = {0:.3f}".format(params[0]))
print("q = {0:.3f}".format(params[1]))
print("F_NR = {0:.3f}".format(params[2]))
print("f_ER = {0:.3f}".format(params[3]))
print("f_NR = {0:.3f}".format(params[4]))
print("f_ng = {0:.3f}\n".format(params[5]))

print("VECTORS")
print("------------")

print("N_meas:",N_meas[3:10])
print("N_pred:",N_pred[:5])
print("dN_meas[0]:",dN_meas[0][3:10])
print("dN_meas[1]:",dN_meas[1][3:10])

print("COMMENT:")
print("Parameters from Nick's version of MCMC calculations.\n") #Any comment you want to be printed along with the data

MODEL: Sor 

STATISTICS
------------
Log likelihood: 1215.1807529030273
Chisq: 376.6743751181112
Chisq/DoF: 1.9929861117360381 

PARAMETERS
------------
k = 0.152
q = 0.002
F_NR = 4.306
f_ER = 4.184
f_NR = 5.513
f_ng = 3.583

VECTORS
------------
N_meas: [       inf 0.00127857 0.03505082 0.03094516 0.03166483 0.02930195
 0.02763481]
N_pred: [0.3768152  0.2869549  0.12631236 0.05402656 0.03726516]
dN_meas[0]: [       nan 0.00129908 0.00084608 0.00072327 0.0006912  0.00064179
 0.00061017]
dN_meas[1]: [       nan 0.00129908 0.00084608 0.00072327 0.0006912  0.00064179
 0.00061017]
COMMENT:
Parameters from Nick's version of MCMC calculations.

